In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
import numpy as np 
from tqdm import tqdm
import time
import random
import networkx as nx
import re

In [4]:
%cd ..

D:\Course\SBRS


In [ ]:
file_data = 'datasets/onlineretail/online_retail.xlsx'
df = pd.read_excel(file_data)

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df = df[['transactionid','itemid', 'userid']]
df.fillna(0, inplace=True)
df['userid'] = df['userid'].astype('int64')
df['itemid'] = df['itemid'].astype('str')
df.head()


In [ ]:
df.transactionid.unique().shape

In [ ]:
list_df = []
for i in tqdm(df.transactionid.unique()[:1001]):
    time.sleep(0.00000001)
    list_df.append(df[df.transactionid == i].itemid.to_list())
list_df

In [ ]:
tr = TransactionEncoder()
tr_arr = tr.fit(list_df).transform(list_df)
df_trans = pd.DataFrame(tr_arr, columns=tr.columns_)

In [ ]:
df_trans.shape

In [ ]:
frequent_itemsets = apriori(df_trans, min_support = 0.005, use_colnames = True, low_memory=False)
frequent_itemsets

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.005)
rules

In [ ]:
rules.iloc[9998].antecedents.issubset(('20724','22384')) # проверка наличия подмножетсва в записи

In [ ]:
def draw_network(rules, rules_to_show):
  # Ориентированный граф от NetworkX
  network = nx.DiGraph()
  
  # Перебор количества правил для отображения
  for i in range(rules_to_show):
    
    # Добавление узла правила
    network.add_nodes_from(["R"+str(i)])
    for antecedents in rules.iloc[i]['antecedents']: 
        # Добавление узла-предшественника и ссылки на правило
        network.add_nodes_from([antecedents])
        network.add_edge(antecedents, "R"+str(i),  weight = 2)
      
    for consequents in rules.iloc[i]['consequents']:
        # Добавление узла-последователя и ссылки на правило
        network.add_nodes_from([consequents])
        network.add_edge("R"+str(i), consequents,  weight = 2)

  color_map=[]  
  
  # Для каждого узла, если это правило, цвет черный, в противном случае - оранжевый.
  for node in network:
       if re.compile("^[R]\d+$").fullmatch(node) != None:
            color_map.append('black')
       else:
            color_map.append('orange')
  
  # Расположение узлов с помощью spring layout
  pos = nx.spring_layout(network, k=16, scale=1)
  # Построение сетевого графа
  nx.draw(network, pos, node_color = color_map, font_size=8)            
  
  # Сдвиг расположения текста вверх
  for p in pos:  
      pos[p][1] += 0.12

  nx.draw_networkx_labels(network, pos)
  plt.title("Network Graph for Association Rules")
  plt.show()

draw_network(rules, 10)

In [ ]:
test_session = list_df[random.randint(0, len(list_df))]
test_session

In [ ]:
k = frozenset({'2021', '20201'})
tuple(list(k))

In [ ]:
# наследник + его оценка (conviction, lift)
def foo(dataframe, session):
    list_of_rank = rules[rules.antecedents == frozenset([str(session)])][['consequents','conviction']].\
                            sort_values(['conviction'], ascending=False).\
                            values.\
                            tolist()
    return list_of_rank

In [ ]:
foo(rules, 22726)

In [ ]:
# на большом наборе данных работает просто ужасно + мы не учитываем множества
def MRR(test_session):
    sum = 0
    len_test_session = len(test_session)-1
    for i in range(len(test_session)-1):
        temp = foo(rules, test_session[i])
        next_item = test_session[i+1]
        for j in range(len(temp)):
            if frozenset([next_item]) == temp[j][0]:
                sum += 1 / (j+1)
                break
        else:
            sum += 0
    return round(sum/len_test_session,3)
MRR(test_session)

In [2]:
1/((1/2 + 1/46 + 1/323 + 1/39 + 1/28)/13)

22.17709403377303